
# Battle of Neighbourhoods - Delhi

##Introduction

In this project we will be analizing the neighbourhoods on the basis of the metro tranis stations located in the city of Delhi.

Later on in the project K Means is used to cluster the data.

In [ ]:
!pip install folium

Here the locations and names of the metro station are loaded into a dataframe from a JSON file. The data is refered from https://raw.githubusercontent.com/dhirajt/delhi-metro-stations/master/metro.json, I would like to thank 'dhirajit' for creating the data and publishing it on opensource GitHub for others to use the data. 

In [23]:
from bs4 import BeautifulSoup
import json
import requests
import pandas as pd
import folium

from pandas.io.json import json_normalize

# Create URL to JSON file (alternatively this can be a filepath)
url = 'https://raw.githubusercontent.com/dhirajt/delhi-metro-stations/master/metro.json'
# Load the first sheet of the JSON file into a data frame
df = pd.read_json(url, orient='columns')

#flat = flatten_json(df)
df_names = df[['name']]
df1 = json_normalize(df['details'])
df = pd.concat([df_names,df1] , axis = 1)

df = df[['name','latitude','longitude','layout','line']]
df.head()
#df.dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137 entries, 0 to 136
Data columns (total 5 columns):
name         137 non-null object
latitude     137 non-null float64
longitude    137 non-null float64
layout       137 non-null object
line         137 non-null object
dtypes: float64(2), object(3)
memory usage: 5.4+ KB


The data from the dataframe is used to visualize it in a map.

In [24]:

import folium 


map_delhi_metro = folium.Map(location =[28.71642,77.17046],zoom_start = 9)


for i in range(0,len(df)):
    folium.Marker([df.iloc[i]['latitude'], df.iloc[i]['longitude']], popup=df.iloc[i]['name'],fill_color='#43d9de', radius=8).add_to(map_delhi_metro)
  

map_delhi_metro


The cell is hidden for the purpose of the credentials. Using the credentials to log to FourSquare api



In [50]:
# @hidden_cell
CLIENT_ID = '4QM3BHOD2W5HXIRA42STOI0GZ4ZQSGU1QDQ3NGF1020X1IOU' # your Foursquare ID
CLIENT_SECRET = 'UQIB4VQTDKX4S04YJ0MPHLL50E0DBPNF3DMSHGTS55I10EEV' # your Foursquare Secret
VERSION = '20180604' # Foursquare API version

In [26]:
categories_url = 'https://api.foursquare.com/v2/venues/categories?client_id={}&client_secret={}&v={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION)
            
# make the GET request
results = requests.get(categories_url).json()

In [27]:

len(results['response']['categories'])

10

In [28]:
categories_list = []
# Let's print only the top-level categories and their IDs and also add them to categories_list

def print_categories(categories, level=0, max_level=0):    
    if level>max_level: return
    out = ''
    out += '-'*level
    for category in categories:
        print(out + category['name'] + ' (' + category['id'] + ')')
        print_categories(category['categories'], level+1, max_level)
        categories_list.append((category['name'], category['id']))
        
print_categories(results['response']['categories'], 0, 0)


Arts & Entertainment (4d4b7104d754a06370d81259)
College & University (4d4b7105d754a06372d81259)
Event (4d4b7105d754a06373d81259)
Food (4d4b7105d754a06374d81259)
Nightlife Spot (4d4b7105d754a06376d81259)
Outdoors & Recreation (4d4b7105d754a06377d81259)
Professional & Other Places (4d4b7105d754a06375d81259)
Residence (4e67e38e036454776db1fb3a)
Shop & Service (4d4b7105d754a06378d81259)
Travel & Transport (4d4b7105d754a06379d81259)


In [29]:
def get_venues_count(lat, lon, radius, categoryId):
    explore_url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={}&radius={}&categoryId={}'.format(
                CLIENT_ID, 
                CLIENT_SECRET, 
                VERSION,
                lat,
                lon,
                radius,
                categoryId)

    # make the GET request
    return requests.get(explore_url).json()['response']['groups'][0]['items']

In [30]:
stations_venues_df = df.copy()
for c in categories_list:
    stations_venues_df[c[0]] = 0
    
stations_venues_df

,name,latitude,longitude,layout,line,Arts & Entertainment,College & University,Event,Food,Nightlife Spot,Outdoors & Recreation,Professional & Other Places,Residence,Shop & Service,Travel & Transport
0,Adarsh Nagar,28.71642,77.17046,Underground,[Yellow Line],0,0,0,0,0,0,0,0,0,0
1,AIIMS,28.56892,77.20771,Underground,[Yellow Line],0,0,0,0,0,0,0,0,0,0
2,Akshardham,28.61806,77.27869,Elevated,[Blue Line],0,0,0,0,0,0,0,0,0,0
3,Anand Vihar,28.64695,77.31603,Elevated,[Blue Line branch],0,0,0,0,0,0,0,0,0,0
4,Arjan Garh,28.48076,77.12583,Elevated,[Yellow Line],0,0,0,0,0,0,0,0,0,0
5,Ashok Park Main,28.67153,77.15527,Elevated,[Green Line],0,0,0,0,0,0,0,0,0,0
6,Azadpur,28.70696,77.18053,Underground,[Yellow Line],0,0,0,0,0,0,0,0,0,0
7,Badarpur,28.49334,77.30307,Elevated,[Violet Line],0,0,0,0,0,0,0,0,0,0
8,Barakhambha Road,28.63003,77.22436,Underground,[Blue Line],0,0,0,0,0,0,0,0,0,0
9,Botanical Garden,28.56409,77.33420,Elevated,[Blue Line],0,0,0,0,0,0,0,0,0,0


In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        LIMIT = 100    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

The Longitude and Latitude values are used to search the neighbourhoods upto 100 meters near the metro stations

In [32]:
toronto_venues = getNearbyVenues(names=stations_venues_df['name'],latitudes=stations_venues_df['latitude'],longitudes=stations_venues_df['longitude'])
print(toronto_venues.shape)
toronto_venues.head()

Adarsh Nagar
AIIMS
Akshardham
Anand Vihar
Arjan Garh
Ashok Park Main
Azadpur
Badarpur
Barakhambha Road
Botanical Garden
Central Secretariat
Chandni Chowk
Chhatarpur
Chawri Bazar
Civil Lines
Delhi Aerocity
Dhaula Kuan
Dilshad Garden
Dwarka
Dwarka Morh
Dwarka Sector 10
Dwarka Sector 11
Dwarka Sector 12
Dwarka Sector 13
Dwarka Sector 14
Dwarka Sector 21
Dwarka Sector 8
Dwarka Sector 9
Ghitorni
Govind Puri
Green Park
GTB Nagar
Guru Dronacharya
Hauz Khas
HUDA City Centre
IFFCO Chowk
INA
Inderlok
Indira Gandhi International Airport
Indraprastha
Jahangirpuri
Janakpuri East
Janakpuri West
Jangpura
Jasola Apollo
Jawaharlal Nehru Stadium
Jhandewalan
Jhilmil
Jor Bagh
Kailash Colony
Kalkaji Mandir
Kanhiya Nagar
Karkarduma
Karol Bagh
Kashmere Gate
Kaushambi
Keshav Puram
Khan Market
Kirti Nagar
Kohat Enclave
Lajpat Nagar
Laxmi Nagar
Madipur
Malviya Nagar
Mandi House
Mansarovar Park
Mayur Vihar -I
Mayur Vihar Extension
MG Road
Model Town
Mohan Estate
Moolchand
Moti Nagar
Mundka
Nangloi
Nangloi Railwa

KeyError: 'groups'

In [33]:
toronto_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
AIIMS,6,6,6,6,6,6
Adarsh Nagar,4,4,4,4,4,4
Akshardham,6,6,6,6,6,6
Anand Vihar,6,6,6,6,6,6
Arjan Garh,5,5,5,5,5,5
Ashok Park Main,1,1,1,1,1,1
Azadpur,3,3,3,3,3,3
Badarpur,2,2,2,2,2,2
Barakhambha Road,19,19,19,19,19,19


In [34]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.shape

toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.shape


(135, 165)

In [35]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AIIMS,Light Rail Station,Restaurant,Airport Food Court,Market,Memorial Site,Indian Restaurant,Donut Shop,Falafel Restaurant,Electronics Store,Eastern European Restaurant
1,Adarsh Nagar,Light Rail Station,Ice Cream Shop,Train Station,Indian Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
2,Akshardham,Dessert Shop,North Indian Restaurant,Hindu Temple,Restaurant,Convenience Store,Metro Station,Diner,Falafel Restaurant,Electronics Store,Eastern European Restaurant
3,Anand Vihar,Clothing Store,Café,Shopping Mall,Food Court,Indian Restaurant,Multiplex,Concert Hall,Comfort Food Restaurant,Falafel Restaurant,Electronics Store
4,Arjan Garh,Hotel,Yoga Studio,Furniture / Home Store,Light Rail Station,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant


Using K Means to clusrter the data

In [37]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 3, 0, 2, 1, 0, 1, 0, 2], dtype=int32)

In [38]:

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='name')

toronto_merged # check the last columns!

,name,latitude,longitude,layout,line,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,28.71642,77.17046,Underground,[Yellow Line],0.0,Light Rail Station,Ice Cream Shop,Train Station,Indian Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
1,AIIMS,28.56892,77.20771,Underground,[Yellow Line],0.0,Light Rail Station,Restaurant,Airport Food Court,Market,Memorial Site,Indian Restaurant,Donut Shop,Falafel Restaurant,Electronics Store,Eastern European Restaurant
2,Akshardham,28.61806,77.27869,Elevated,[Blue Line],3.0,Dessert Shop,North Indian Restaurant,Hindu Temple,Restaurant,Convenience Store,Metro Station,Diner,Falafel Restaurant,Electronics Store,Eastern European Restaurant
3,Anand Vihar,28.64695,77.31603,Elevated,[Blue Line branch],0.0,Clothing Store,Café,Shopping Mall,Food Court,Indian Restaurant,Multiplex,Concert Hall,Comfort Food Restaurant,Falafel Restaurant,Electronics Store
4,Arjan Garh,28.48076,77.12583,Elevated,[Yellow Line],2.0,Hotel,Yoga Studio,Furniture / Home Store,Light Rail Station,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
5,Ashok Park Main,28.67153,77.15527,Elevated,[Green Line],1.0,Train Station,Yoga Studio,Diner,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
6,Azadpur,28.70696,77.18053,Underground,[Yellow Line],0.0,Bus Station,Indian Restaurant,Park,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop,Discount Store
7,Badarpur,28.49334,77.30307,Elevated,[Violet Line],1.0,Train Station,Asian Restaurant,Yoga Studio,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
8,Barakhambha Road,28.63003,77.22436,Underground,[Blue Line],0.0,Indian Restaurant,Bar,Salon / Barbershop,Café,Portuguese Restaurant,Bistro,Beer Garden,Donut Shop,BBQ Joint,Asian Restaurant
9,Botanical Garden,28.56409,77.33420,Elevated,[Blue Line],2.0,Bus Station,Clothing Store,Hotel,Café,Pizza Place,Metro Station,Bakery,Indian Restaurant,Flea Market,Gastropub


Visualizing the Clusters

In [39]:
import matplotlib.cm as cm
import matplotlib.colors as colors


# create map
map_clusters = folium.Map(location =[28.71642,77.17046],zoom_start = 9)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

toronto_merged_nonan = toronto_merged.dropna(subset=['Cluster Labels'])

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_nonan['latitude'], toronto_merged_nonan['longitude'], toronto_merged_nonan['name'], toronto_merged_nonan['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1 

In [42]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 0, toronto_merged_nonan.columns[[0] + list(range(5, toronto_merged_nonan.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adarsh Nagar,0.0,Light Rail Station,Ice Cream Shop,Train Station,Indian Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
1,AIIMS,0.0,Light Rail Station,Restaurant,Airport Food Court,Market,Memorial Site,Indian Restaurant,Donut Shop,Falafel Restaurant,Electronics Store,Eastern European Restaurant
3,Anand Vihar,0.0,Clothing Store,Café,Shopping Mall,Food Court,Indian Restaurant,Multiplex,Concert Hall,Comfort Food Restaurant,Falafel Restaurant,Electronics Store
6,Azadpur,0.0,Bus Station,Indian Restaurant,Park,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop,Discount Store
8,Barakhambha Road,0.0,Indian Restaurant,Bar,Salon / Barbershop,Café,Portuguese Restaurant,Bistro,Beer Garden,Donut Shop,BBQ Joint,Asian Restaurant
11,Chandni Chowk,0.0,Indian Restaurant,Fast Food Restaurant,Market,Snack Place,Dessert Shop,Restaurant,Flea Market,Historic Site,Train Station,Convenience Store
12,Chhatarpur,0.0,Flea Market,Indian Restaurant,Japanese Restaurant,Public Art,Metro Station,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
17,Dilshad Garden,0.0,Diner,Pizza Place,Light Rail Station,Indian Restaurant,Yoga Studio,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
19,Dwarka Morh,0.0,Fast Food Restaurant,Pizza Place,Light Rail Station,Indian Restaurant,Diner,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
29,Govind Puri,0.0,Beer Garden,Train Station,Park,Fast Food Restaurant,Indian Restaurant,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store


Cluster 2

In [43]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 1, toronto_merged_nonan.columns[[0] + list(range(5, toronto_merged_nonan.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Ashok Park Main,1.0,Train Station,Yoga Studio,Diner,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
7,Badarpur,1.0,Train Station,Asian Restaurant,Yoga Studio,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
51,Kanhiya Nagar,1.0,Clothing Store,Train Station,Athletics & Sports,Light Rail Station,Concert Hall,Comfort Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Electronics Store
52,Karkarduma,1.0,Light Rail Station,Train Station,Food Truck,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
62,Madipur,1.0,Train Station,Hotel,Yoga Studio,Diner,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
73,Mundka,1.0,ATM,Train Station,Business Service,Mobile Phone Shop,Convenience Store,Discount Store,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
89,Paschim Vihar West,1.0,ATM,Train Station,Pizza Place,Women's Store,Airport,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store
95,Pratap Nagar,1.0,Train Station,Mobile Phone Shop,Light Rail Station,Comfort Food Restaurant,Concert Hall,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
98,Punjabi Bagh East,1.0,Gaming Cafe,Restaurant,Train Station,Yoga Studio,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
99,Qutab Minar,1.0,Sculpture Garden,Train Station,Light Rail Station,Metro Station,Concert Hall,Comfort Food Restaurant,Farm,Coffee Shop,Falafel Restaurant,Electronics Store


In [44]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 2, toronto_merged_nonan.columns[[0] + list(range(5, toronto_merged_nonan.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Arjan Garh,2.0,Hotel,Yoga Studio,Furniture / Home Store,Light Rail Station,Donut Shop,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
9,Botanical Garden,2.0,Bus Station,Clothing Store,Hotel,Café,Pizza Place,Metro Station,Bakery,Indian Restaurant,Flea Market,Gastropub
14,Civil Lines,2.0,Indian Restaurant,Chinese Restaurant,Café,Hotel,Burger Joint,Light Rail Station,Asian Restaurant,Donut Shop,Farmers Market,Farm
15,Delhi Aerocity,2.0,Hotel,Indian Restaurant,Bed & Breakfast,Buffet,Coffee Shop,Restaurant,Plaza,Hotel Bar,Lounge,North Indian Restaurant
20,Dwarka Sector 10,2.0,Hotel,Asian Restaurant,Electronics Store,Light Rail Station,Restaurant,Shopping Mall,Yoga Studio,Farm,Falafel Restaurant,Eastern European Restaurant
23,Dwarka Sector 13,2.0,Restaurant,Clothing Store,Hotel,Light Rail Station,Electronics Store,Shopping Mall,Bakery,Discount Store,Indian Restaurant,Asian Restaurant
28,Ghitorni,2.0,Sandwich Place,Pizza Place,Light Rail Station,Indian Restaurant,Hotel,Sporting Goods Shop,Lake,Discount Store,Coffee Shop,Falafel Restaurant
32,Guru Dronacharya,2.0,Coffee Shop,Hotel,Light Rail Station,Chinese Restaurant,Restaurant,French Restaurant,Donut Shop,Farm,Falafel Restaurant,Electronics Store
34,HUDA City Centre,2.0,Hotel,Café,Fast Food Restaurant,Comfort Food Restaurant,Gym,Pizza Place,Donut Shop,Breakfast Spot,Restaurant,Snack Place
39,Indraprastha,2.0,Coffee Shop,Café,Light Rail Station,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop


In [45]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 3, toronto_merged_nonan.columns[[0] + list(range(5, toronto_merged_nonan.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Akshardham,3.0,Dessert Shop,North Indian Restaurant,Hindu Temple,Restaurant,Convenience Store,Metro Station,Diner,Falafel Restaurant,Electronics Store,Eastern European Restaurant
10,Central Secretariat,3.0,Music Venue,Art Gallery,Coffee Shop,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
13,Chawri Bazar,3.0,Hardware Store,Paper / Office Supplies Store,Snack Place,Ice Cream Shop,Concert Hall,Comfort Food Restaurant,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
16,Dhaula Kuan,3.0,Sake Bar,Campground,Golf Course,Bakery,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant
18,Dwarka,3.0,Cosmetics Shop,Metro Station,Yoga Studio,Discount Store,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
21,Dwarka Sector 11,3.0,Ice Cream Shop,Yoga Studio,Diner,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
22,Dwarka Sector 12,3.0,Shopping Mall,Market,Pizza Place,Donut Shop,Farmers Market,Yoga Studio,Discount Store,Falafel Restaurant,Electronics Store,Eastern European Restaurant
24,Dwarka Sector 14,3.0,Light Rail Station,Shopping Mall,Movie Theater,Diner,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
25,Dwarka Sector 21,3.0,Light Rail Station,Farm,Cosmetics Shop,Platform,Tea Room,Diner,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop
27,Dwarka Sector 9,3.0,Burger Joint,Chaat Place,Bar,Train Station,Fast Food Restaurant,Yoga Studio,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant


In [46]:
toronto_merged_nonan.loc[toronto_merged_nonan['Cluster Labels'] == 4, toronto_merged_nonan.columns[[0] + list(range(5, toronto_merged_nonan.shape[1]))]]

,name,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
26,Dwarka Sector 8,4.0,Light Rail Station,Pizza Place,Diner,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
40,Jahangirpuri,4.0,Light Rail Station,Yoga Studio,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop,Discount Store
87,Okhla,4.0,ATM,Light Rail Station,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop,Discount Store
106,Ramesh Nagar,4.0,Light Rail Station,Pizza Place,Shoe Store,Chinese Restaurant,Dessert Shop,Concert Hall,Cocktail Bar,Farm,Falafel Restaurant,Electronics Store
135,Welcome,4.0,Light Rail Station,Shopping Plaza,Discount Store,Farmers Market,Farm,Falafel Restaurant,Electronics Store,Eastern European Restaurant,Duty-free Shop,Donut Shop
